# Find the Tents

_Combinatorial Optimization course, FEE CTU in Prague. Created by [Industrial Informatics Department](http://industrialinformatics.fel.cvut.cz)._

The problem was taken from https://www.brainbashers.com/tents.asp ; there, you can try to solve some examples manually.

## Task

Find all of the hidden tents in the forest grid.

You know that:

- Each tent is attached to one tree (so there are as many tents as there are trees).
- A tent can only be found horizontally or vertically adjacent to a tree.
- Tents are never adjacent to each other, neither vertically, horizontally, nor diagonally.
- A tree might be next to two tents but is only connected to one.

You are also given two vectors indicating how many tents are in each respective row or column of the forest grid.


## Input

You are given a positive integer $n \geq 2$, representing the size of the forest grid (assume it is a square of size $(n \times n$). You are also given vectors $\mathbf r = (r_1, \dots, r_n)$ and $\mathbf c = (c_1, \dots, c_n)$ representing the numbers of the tents in the rows and columns of the forest grid. Finally, you are given a list of coordinates of the trees $((x_1, y_1), \dots, (x_k, y_k))$.

In [87]:
# 2x2 - Extra small (for debugging)
n1 = 3
r1 = (1, 1, 0)
c1 = (1, 0, 1)
trees1 = [(1,1), (3,2)]


In [88]:
# 8x8 - Medium
n2 = 8
r2 = (3, 1, 1, 2, 0, 2, 0, 3)
c2 = (2, 1, 2, 2 ,1, 1 ,2 ,1)
trees2 = [(2, 1), (5, 1), (6, 1),
         (1, 2),
         (3, 3),
         (3, 4), (6, 4),
         (4, 5), (6, 5),
         (8, 7),
         (2, 8), (4, 8)]

In [89]:
# Weekly special
n3 = 20
r3 = (7, 2, 3, 4, 3, 5, 4, 4, 4, 4, 3, 6, 3, 6, 2, 3, 6, 3, 3, 5)
c3 = (6, 4, 3, 5, 4, 4, 4, 3, 5, 3, 4, 3, 4, 4, 6, 3 ,4, 3, 6, 2)
trees3 = [(3, 1), (4, 1), (8, 1), (13, 1), (15, 1),
         (1, 2), (9, 2), (18, 2), (19, 2),
         (5, 3), (12, 3), (15, 3),
         (2, 4), (4, 4), (9, 4), (17, 4),
         (6, 5), (10, 5), (13, 5), (17, 5), (20, 5),
         (1, 6), (7, 6), (10, 6), (12, 6), (16, 6),
         (20, 7),
         (1, 8), (4, 8), (5, 8), (11, 8), (13, 8), (14, 8), (19, 8),
         (4, 9), (6, 9), (9, 9), (15, 9), (17, 9),
         (8, 10), (17, 10), (19, 10),
         (12, 11),
         (5, 12), (7, 12), (14, 12), (16, 12),
         (1, 13), (2, 13), (6, 13), (19, 13),
         (11, 14), (14, 14), (20, 14),
         (3, 15), (5, 15), (6, 15), (8, 15), (13, 15), (20, 15),
         (2, 16), (3, 16), (10, 16),
         (8, 17), (11, 17), (14, 17), (15, 17),
         (2, 18), (6, 18), (9, 18), (12, 18), (13, 18), (18, 18),
         (2, 19), (7, 19), (15, 19), (17, 19), (20, 19),
         (5, 20), (10, 20)]

## Output

You should find the coordinates $(x_i, y_i), i \in \{1,\dots,k\}$, of the individual tents.

## Model

In [90]:
import gurobipy as g
from gurobipy import *

def optimize1(n,r,c,trees):
    def to_xy(index):
        return index%n,int(index/n)

    def to_index(x,y):
        return y*n + x
    
    def offset_4(x,y):
        return [(x+1,y), (x-1,y), (x,y+1), (x,y-1)]
    
    # VARIABLES
    model = g.Model()
    # Pozice stanu
    x = model.addVars(n*n, vtype=GRB.BINARY, name='x')
    
    trees_vector = [0 for _ in range(n*n)]
    for a,b in trees:
        trees_vector[to_index(a-1,b-1)] = 1
    
    # Constraints
    # soucty radku
    for i in range(len(r)):
        value = r[i]
        model.addConstr(sum(x.values()[i*n:i*n+n]) == value)    
    
    # soucty sloupcu
    for i in range(len(c)):
        value = c[i]
        end = i + (n-1)*n + 1
        model.addConstr(sum(x.values()[i:end:n]) == value)    
    
    # V osmi okolí stanu nesmí být stan
    for index, tent in enumerate(x):
        xx,yy = to_xy(index)
        for i in [xx-1,xx,xx+1]:
            for j in [yy-1,yy,yy+1]:
                if 0 <= i < n and 0 <= j < n and (i,j) != (xx,yy):
                   model.addConstr(x[to_index(i,j)] <= 1-x[to_index(xx,yy)])
     
    # stan nesmí být na stromě
    # ve čtyř okolí stanu MUSÍ být strom    
    for index, _ in enumerate(x):
        if trees_vector[index] == 1: # strom
            model.addConstr(x[index] <= 0)
            # ve čtyř okolí stromu musí být stan protože stanů je stejně jako stromů
            model.addConstr(sum([x[to_index(a,b)] for a,b in offset_4(*to_xy(index)) if 0 <= a < n and 0 <= b < n]) >= 1)
        model.addConstr(sum([trees_vector[to_index(a,b)] for a,b in offset_4(*to_xy(index)) if 0 <= a < n and 0 <= b < n]) >= x[index])
                
    
    model.optimize()
    # fill tents
    
    tents = []    
    for index, tent in enumerate(x.values()): 
        if tent.x != 0:
            a,b = to_xy(index)
            tents.append((a+1,b+1))
    return tents


In [91]:
def optimize2(n, r, c, trees):
    import gurobipy as g
    from gurobipy import GRB
    from itertools import product as cartesian
    
    offsets_8 = set(cartesian(range(-1,2), range(-1,2))) - {(0, 0)}
    offsets_4 = [(1,0), (-1,0), (0,1), (0,-1)]
    outer_frame = set(cartesian(range(n+2), range(n+2))) - set(cartesian(range(1,n+1), range(1,n+1)))
    
    m = g.Model()
    T = m.addVars(n+2, n+2, vtype=GRB.BINARY, ub=0)
    
    for i, j in trees:
        tree_four_neighbors = [(i+ii, j+jj) for (ii,jj) in offsets_4]
        for loc in tree_four_neighbors:
            T[loc].ub = 1
        m.addConstr(g.quicksum(T[loc] for loc in tree_four_neighbors) >= 1)
    
    m.addConstr(g.quicksum(T[coords] for coords in outer_frame) == 0)  # can't put tents in fake spots
        
    M = 9
    
    for i, j in cartesian(range(1, n+1), range(1, n+1)):
        # there can't be another tent in a tent's 8-neighborhood
        m.addConstr(M * (1 - T[i, j]) >= g.quicksum(T[i+ii, j+jj] for ii, jj in offsets_8))
        
    for i, val in enumerate(r, start=1):
        m.addConstr(g.quicksum(T[j, i] for j in range(1, n + 1)) == val)
    
    for j, val in enumerate(c, start=1):
        m.addConstr(g.quicksum(T[j, i] for i in range(1, n + 1)) == val)
        
    m.optimize()
    
    return [coords for coords, var in T.items() if var.x > 0]

 ##  Visualization

In [92]:
import matplotlib.pyplot as plt
import numpy as np

def visualize(n, trees, tents, r, c):
    grid = [["." for _ in range(n+2)] for _ in range(n+2)]
    
    for t_x, t_y in tents:
        grid[t_y][t_x] = "X"
    
    for t_x, t_y in trees:
        grid[t_y][t_x] = "T"

    print("  ", end="")
    for c_cur in c:
        print(c_cur, end=" ")
    print()
    
    for y in range(1, n+1):
        print(r[y-1], end=" ")
        for x in range(1, n+1):
            print(grid[y][x], end=" ")
            
        print()

In [93]:
optimize = optimize1  # select ILP model

In [94]:
tents1 = optimize(n1, r1, c1, trees1)
print(tents1)
visualize(n1, trees1, tents1, r1, c1)

Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (win64)
Optimize a model with 59 rows, 9 columns and 114 nonzeros
Model fingerprint: 0x6e0e1d66
Variable types: 0 continuous, 9 integer (9 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 59 rows and 9 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.03 seconds
Thread count was 1 (of 4 available processors)

Solution count 1: 0 

Optimal solution found (tolerance 1.00e-04)
Best objective 0.000000000000e+00, best bound 0.000000000000e+00, gap 0.0000%
[(3, 1), (1, 2)]
  1 0 1 
1 T . X 
1 X . T 
0 . . . 


In [95]:
optimize = optimize2  # select ILP model

Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (win64)
Optimize a model with 524 rows, 64 columns and 1085 nonzeros
Model fingerprint: 0x9167e6f3
Variable types: 0 continuous, 64 integer (64 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+00]
Presolve removed 524 rows and 64 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.02 seconds
Thread count was 1 (of 4 available processors)

Solution count 1: 0 

Optimal solution found (tolerance 1.00e-04)
Best objective 0.000000000000e+00, best bound 0.000000000000e+00, gap 0.0000%
  2 1 2 2 1 1 2 1 
3 X T X . T T X . 
1 T . . . X . . . 
1 . X T . . . . . 
2 . . T X . T X . 
0 . . . T . T . . 
2 . . . X . X . . 
0 . . . . . . . T 
3 X T X T . . . X 


In [96]:
tents1 = optimize(n1, r1, c1, trees1)
visualize(n1, trees1, tents1, r1, c1)

Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (win64)
Optimize a model with 3564 rows, 400 columns and 7512 nonzeros
Model fingerprint: 0xfbd745af
Variable types: 0 continuous, 400 integer (400 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 7e+00]
Presolve removed 3564 rows and 400 columns
Presolve time: 0.01s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.02 seconds
Thread count was 1 (of 4 available processors)

Solution count 1: 0 

Optimal solution found (tolerance 1.00e-04)
Best objective 0.000000000000e+00, best bound 0.000000000000e+00, gap 0.0000%
  6 4 3 5 4 4 4 3 5 3 4 3 4 4 6 3 4 3 6 2 
7 . X T T X . X T X . . . T X T X . X . . 
2 T . . . . . . . T . . X . . . . . T T X 
3 X . . X T . . . . . . T . X T . . . . . 
4 . T . T . X . X T X . . . . . X T . . . 
3 . X . X . T . . . T . . T . . . T . X T 
5 T . . . . . T . X T X T X

In [16]:
tents2 = optimize(n2, r2, c2, trees2)
visualize(n2, trees2, tents2, r2, c2)

Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (win64)
Optimize a model with 93 rows, 100 columns and 788 nonzeros
Model fingerprint: 0x3db04752
Variable types: 0 continuous, 100 integer (100 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+00]
  Objective range  [0e+00, 0e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 9e+00]
Found heuristic solution: objective 0.0000000

Explored 0 nodes (0 simplex iterations) in 0.01 seconds
Thread count was 1 (of 12 available processors)

Solution count 1: 0 

Optimal solution found (tolerance 1.00e-04)
Best objective 0.000000000000e+00, best bound 0.000000000000e+00, gap 0.0000%
  2 1 2 2 1 1 2 1 
3 X T X . T T X . 
1 T . . . X . . . 
1 . X T . . . . . 
2 . . T X . T X . 
0 . . . T . T . . 
2 . . . X . X . . 
0 . . . . . . . T 
3 X T X T . . . X 


In [17]:
tents3 = optimize(n3, r3, c3, trees3)
visualize(n3, trees3, tents3, r3, c3)


Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (win64)
Optimize a model with 521 rows, 484 columns and 4804 nonzeros
Model fingerprint: 0x329e6931
Variable types: 0 continuous, 484 integer (484 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+00]
  Objective range  [0e+00, 0e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 9e+00]
Presolve removed 200 rows and 283 columns
Presolve time: 0.01s
Presolved: 321 rows, 201 columns, 1577 nonzeros
Variable types: 0 continuous, 201 integer (201 binary)

Root relaxation: objective 0.000000e+00, 486 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

H    0     0                       0.0000000    0.00000  0.00%     -    0s
     0     0    0.00000    0   69    0.00000    0.00000  0.00%     -    0s

Explored 0 nodes (744 simplex iterations) in 0.05 seconds
Thread count was 12 (of 12 available pro